# Functional Test 3.1.1 - creating simple VMs of varying sizes

This Jupyter notebook will allow you to create VMs of different sizes on different sites and worker nodes consistent with requirements for test 3.1.1.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

-----------------------------------  --------------------------------------------------
credmgr_host                         cm.fabric-testbed.net
orchestrator_host                    orchestrator.fabric-testbed.net
fabric_token                         /home/fabric/.tokens.json
project_id                           990d8a8b-7e50-4d13-a3be-0f133ffa8653
bastion_username                     ibaldin_0000241998
bastion_key_filename                 /home/fabric/work/fabric_config/fabric_bastion_key
bastion_public_addr                  bastion-1.fabric-testbed.net
bastion_passphrase                   None
slice_public_key_file                /home/fabric/work/fabric_config/slice_key.pub
slice_private_key_file               /home/fabric/work/fabric_config/slice_key
fabric_slice_private_key_passphrase  None
fablib_log_file                      /tmp/fablib/fablib.log
fablib_log_level                     INFO
-----------------------------------  --------------------------------------------------


## Step 3 Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [16]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4 Query for Available Testbed Resources and Settings

This command queries the FABRIC services to find the available resources. 

**Be sure the site you are about to test is listed in the advertisement.**

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
MASS         4  125/128  1016/1024  55750/55800    253/254                 2/2                             0/0                            6/6                 0/0               3/3
MAX         10  264/320  2402/2560  115468/116400  610/635                 0/2                             1/4                            16/16               4/4               6/6
TACC        10  306/320  2516/2560  116350/116400  633/635                 0/2                             1/4                            16/16               4/4               6/6
MICH         6  192/192  1536/1536  60600/60600    381/381                 2/2

## Step 5: Create the test Slice

The following creates a single node with basic compute capabilities at a specific site on a specific worker. Modify the site, worker and the dimensions of the desired VM. 

**RAM and disk are measured in GB.**

**Note that FABRIC should give you a larger VM than you are asking up to a maximum limit.** 

**Pay attention to how many cores, RAM and Disk are actually allocated.**

**The code to create the slice will auto-refresh until the slice is created or it fails**

In [24]:
from datetime import datetime
from dateutil import tz

name='Node1'
site='TACC'
# since all workers have a standard naming scheme, you can just change the worker
# to move from worker to worker
worker=f'{site.lower()}-w1.fabric-testbed.net'
cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.1 {site} on {worker} on {datetime.now()}"

In [20]:
try:
    # Create a slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Add a node
    slice.add_node(name=name, site=site, host=worker, cores=cores, ram=ram, disk=disk)
    
    # Submit the slice
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


-----------  ---------------------------------------------------------------------------
Slice Name   Slice Test TACC on tacc-w1.fabric-testbed.net on 2022-08-31 00:17:56.594589
Slice ID     86b81b23-c107-4252-89e5-5623a0cf5bf9
Slice State  StableOK
Lease End    2022-09-01 00:18:29 +0000
-----------  ---------------------------------------------------------------------------

Retry: 8, Time: 91 sec

ID                                    Name    Site    Host                          Cores    RAM    Disk  Image            Management IP    State    Error
------------------------------------  ------  ------  --------------------------  -------  -----  ------  ---------------  ---------------  -------  -------
50783434-8f1c-4749-a67d-4757cb2d1962  Node1   TACC    tacc-w1.fabric-testbed.net       10     32     100  default_rocky_8  129.114.110.74   Active

Time to stable 91 seconds
Running post_boot_config ... Time to post boot config 92 seconds


## Step 6: Observe the Slice's Attributes

### Print the slice 

In [21]:
try:
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

-----------  ---------------------------------------------------------------------------
Slice Name   Slice Test TACC on tacc-w1.fabric-testbed.net on 2022-08-31 00:17:56.594589
Slice ID     86b81b23-c107-4252-89e5-5623a0cf5bf9
Slice State  StableOK
Lease End    2022-09-01 00:18:29 +0000
-----------  ---------------------------------------------------------------------------


### Print the node

Each node in the slice has a set of get functions that return the node's attributes. Use the returned `SSH Command` string to check the node. You can do it from a Bash launched inside the Jupyter container.

In [22]:
try:
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

-----------------  ------------------------------------------------------------------------------------------------------------------------
ID                 50783434-8f1c-4749-a67d-4757cb2d1962
Name               Node1
Cores              10
RAM                32
Disk               100
Image              default_rocky_8
Image Type         qcow2
Host               tacc-w1.fabric-testbed.net
Site               TACC
Management IP      129.114.110.74
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -J ibaldin_0000241998@bastion-1.fabric-testbed.net rocky@129.114.110.74
-----------------  ------------------------------------------------------------------------------------------------------------------------


## Step 7: Delete the Slice

Please delete your slice when you are done with your experiment.

In [23]:
try:
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")